# Panda-Gym con Stable-baseline3 🤖


El objetivo es usar los algoritmos ya implementados para el entrenamiento de entornos, en esta ocasión se usará el entorno de [Panda-Gym](https://github.com/qgallouedec/panda-gym). Entrenarás un **brazo robótico** (el robot Franka Emika Panda) para realizar un tarea.

- `Reach`: el robot debe colocar su efector final en una posición objetivo.

Después de eso, podrás **entrenar en otras tareas robóticas**.

### 🎮 Entorno:

- [Panda-Gym](https://github.com/qgallouedec/panda-gym)

###📚 Libreria RL:

- [Stable-Baselines3](https://stable-baselines3.readthedocs.io/)

## Objectivos 🏆

* Utilizar un entorno robótico como panda-gym
* Emplear una librería de algoritmos de aprendizaje por refuerzo
* Tips para la implementación de los algoritmos




# Configuración 🤖

## Crear una pantalla virtual 🔽

Durante la ejecución, necesitaremos generar un video de repetición. Para hacerlo, en colab, **necesitamos tener una pantalla virtual para poder renderizar el entorno** (y así grabar los fotogramas).

Por lo tanto, la siguiente celda instalará las librerías, creará y ejecutará una pantalla virtual 🖥.

In [1]:
%%capture
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [2]:
# monitor virtual
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

### Instalar dependencias 🔽

El primer paso es instalar las dependencias, instalaremos varias:
- `gymnasium`
- `panda-gym`: Contiene los entornos del brazo robótico.
- `stable-baselines3`: La biblioteca de aprendizaje profundo de refuerzo SB3.

⏲ La instalación puede **tardar 10 minutos**.

In [3]:
!pip install stable-baselines3[extra]
!pip install gymnasium
!pip install panda_gym

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
import os

import gymnasium as gym
import panda_gym

from stable_baselines3 import DDPG
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env


## PandaReachDense-v3 🦾

El agente que vamos a entrenar es un brazo robótico que necesita realizar controles (mover el brazo y utilizar el efector final).

En robótica, el *efector final* es el dispositivo en el extremo de un brazo robótico diseñado para interactuar con el entorno.

En `PandaReach`, el robot debe colocar su efector final en una posición objetivo (bola verde). [Otros entornos de panda-gym](https://panda-gym.readthedocs.io/en/latest/usage/environments.html)

Vamos a usar la versión densa de este entorno. Esto significa que tendremos una *función de recompensa densa* que **proporcionará una recompensa en cada paso de tiempo** (cuanto más cerca esté el agente de completar la tarea, mayor será la recompensa). A diferencia de una *función de recompensa escasa*, donde el entorno **solo devuelve una recompensa si y solo si la tarea se completa**.

También vamos a usar el *control de desplazamiento del efector final*, lo que significa que la **acción corresponde al desplazamiento del efector final**. No controlamos el movimiento individual de cada articulación (control por articulación).

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcR90do7v-58OGvPW4Gyd2ZNSjw4BJIzu8Qi-w&s" />

De esta manera **el entrenamiento será más fácil**.

In [5]:
env_id = "PandaReachDense-v3"

# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape
a_size = env.action_space

El espacio de observación **es un diccionario con 3 elementos diferentes**:
- `achieved_goal`: (x,y,z) la posición actual del efector final.
- `desired_goal`: (x,y,z) la posición objetivo para el efector final.
- `observation`: posición (x,y,z) y velocidad del efector final (vx, vy, vz).

Dado que es un diccionario como observación, **necesitaremos usar una política MultiInputPolicy en lugar de MlpPolicy**.

El espacio de acción es un vector con 3 valores:
- Control del movimiento en x, y, z.

In [6]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation {'achieved_goal': array([-0.75458986,  1.0464022 ,  3.1721694 ], dtype=float32), 'desired_goal': array([ 9.026851,  1.2366  , -4.128559], dtype=float32), 'observation': array([ 8.380305  ,  5.9903316 , -8.513405  , -9.195543  , -4.827909  ,
        0.29232147], dtype=float32)}

 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [0.41930375 0.051084   0.36307752]


In [7]:
import matplotlib.pyplot as plt
import imageio

# Configurar la pantalla virtual
display = Display(visible=0, size=(1400, 900))
display.start()

# Crear el entorno
env = gym.make("PandaReachDense-v3", render_mode="rgb_array")
obs = env.reset()

# Lista para almacenar los fotogramas
frames = []

# Ejecutar pasos y guardar fotogramas
for step in range(50):  # Número de pasos
    action = env.action_space.sample()  # Tomar acciones aleatorias
    obs, reward, terminated, truncated, info = env.step(action)  # Hacer un paso en el entorno

    # Renderizar y guardar el fotograma
    img = env.render()
    frames.append(img)

    if terminated or truncated:  # Reiniciar el entorno si se alcanza el objetivo
        obs = env.reset()

# Cerrar el entorno
env.close()

# Guardar los fotogramas como GIF
gif_path = "panda_gym_render.gif"
imageio.mimsave(gif_path, frames, fps=10)
print(f"GIF guardado con éxito: {gif_path}")

/usr/local/lib/python3.11/dist-packages/imageio/plugins/pillow.py:409: DeprecationWarning: The keyword `fps` is no longer supported. Use `duration`(in ms) instead, e.g. `fps=50` == `duration=20` (1000 * 1/50).
  warnings.warn(


GIF guardado con éxito: panda_gym_render.gif


### Normalizar las observaciones y recompensas

Una buena práctica en el aprendizaje por refuerzo es [normalizar las características de entrada](https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html) añadiendo `norm_obs = True`.

Para ello, existe un wrapper que calculará un promedio móvil y la desviación estándar de las características de entrada.

También normalizamos las recompensas con este mismo wrapper añadiendo `norm_reward = True`.

[Deberías consultar la documentación para completar esta celda](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecnormalize).

In [ ]:
# make_vec_env es parte de stable-baseline3
env = make_vec_env(env_id, n_envs=4)

# Agrega aqui la instrucción para normalizar las observaciones y la recompensa
env = # TODO: Add the wrapper

#### Solution

In [8]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

### Create un agente DDPG con SBL3 🤖

Para más información sobre la implementación de DDPG con StableBaselines3, consulta: https://stable-baselines3.readthedocs.io/en/master/modules/ddpg.html#notes

Para encontrar los mejores parámetros, revisé los [agentes entrenados oficialmente por el equipo de Stable-Baselines3](https://huggingface.co/sb3).

In [ ]:
agent = # Crea un agente DDPG, con politica MultiInputPolicy

#### Solution

In [9]:
agent = DDPG(policy = "MultiInputPolicy",
            env = env,
            verbose=1)

Using cpu device


### Entrenamiento del agente 🏃
- Vamos entrenar el agente por 1000,000 de pasos, si puedes usa colab con GPU. Tardará alrededor de 25~40min

In [10]:
agent.learn(1000_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 43.2     |
|    ep_rew_mean     | -11.9    |
|    success_rate    | 0.25     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 126      |
|    time_elapsed    | 1        |
|    total_timesteps | 200      |
| train/             |          |
|    actor_loss      | 0.18     |
|    critic_loss     | 0.00142  |
|    learning_rate   | 0.001    |
|    n_updates       | 24       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 46.6     |
|    ep_rew_mean     | -24.2    |
|    success_rate    | 0.125    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 102      |
|    time_elapsed    | 3        |
|    total_timesteps | 400      |
| train/             |          |
|    actor_loss      | 0.132    |
|    critic_loss     | 0.000251 |
|    learning_

In [11]:
# Salva el modelo y los parámetros de normalización en el drive, para usarlos después.
#agent.save("/content/drive/mydrive/colab notebooks/DDPG_agent")
#env.save("/content/drive/mydrive/colab notebooks/DDPG_vec_normalize.pkl")

# Salva el modelo y los parámetros de normalización en el colab actual, estos se borrarán al terminar sesión.
agent.save("ddpg-PandaReachDense-v3")
env.save("vec_normalize.pkl")

### Evaluar el agente 📈
- Ya con el agente entrenado necesitamos **revisar sus desempeño**.
- Stable-Baselines3 tiene un método para revisar el agente: `evaluate_policy`

In [13]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# carga los parámetros de normalización
eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

# Se necesita el render rgb_array para colab
eval_env.render_mode = "rgb_array"

#  Deshabilita el entrenamiento
eval_env.training = False
# No se requiere normalización de la recompensa ya que no se esta entrenando
eval_env.norm_reward = False

# Carga el agente
agent = DDPG.load("ddpg-PandaReachDense-v3")

mean_reward, std_reward = evaluate_policy(agent, eval_env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Mean reward = -26.90 +/- 8.00


In [20]:
# Crear el entorno
#env = gym.make("PandaReachDense-v3", render_mode="rgb_array")
obs = eval_env.reset()

# Lista para almacenar los fotogramas
frames = []

# Ejecutar pasos y guardar fotogramas
for step in range(50):  # Número de pasos
    action, _ = agent.predict(obs, deterministic=True)  # Tomar acciones del agente
    obs, reward, done, info = eval_env.step(action)  # Hacer un paso en el entorno

    # Renderizar y guardar el fotograma
    img = eval_env.render()
    frames.append(img)

    if done:  # Reiniciar el entorno si se alcanza el objetivo
        obs = eval_env.reset()

# Cerrar el entorno
eval_env.close()

# Guardar los fotogramas como GIF
gif_path = "panda_gym_render_trained.gif"
imageio.mimsave(gif_path, frames, fps=10)
print(f"GIF guardado con éxito: {gif_path}")

/usr/local/lib/python3.11/dist-packages/imageio/plugins/pillow.py:409: DeprecationWarning: The keyword `fps` is no longer supported. Use `duration`(in ms) instead, e.g. `fps=50` == `duration=20` (1000 * 1/50).
  warnings.warn(


GIF guardado con éxito: panda_gym_render_trained.gif
